In [20]:
pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


# Import Library

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense


- andas dan numpy digunakan untuk manipulasi data.

- MinMaxScaler dari sklearn digunakan untuk normalisasi data.

- Sequential, LSTM, dan Dense dari Keras digunakan untuk membangun dan melatih model LSTM.

- joblib bisa digunakan untuk menyimpan scaler jika diperlukan.

# Load Dataset

In [2]:
# Load dataset
df = pd.read_csv('Data Garjas Unhan - Sheet1.csv')
df.head()

,ID,NAMA,BULAN,PUSH UP,PULL UP,LARI 12 MENIT,SIT UP,SHUTTLE RUN
0,P001,A .MUH. RIFAT ATHO ULLAH,15/12/2024,39,18,2905,39,17.31
1,P001,A .MUH. RIFAT ATHO ULLAH,17/01/2025,40,18,2900,39,17.57
2,P001,A .MUH. RIFAT ATHO ULLAH,14/02/2025,39,18,2960,39,17.23
3,P001,A .MUH. RIFAT ATHO ULLAH,15/03/2025,42,18,2910,39,18.08
4,P001,A .MUH. RIFAT ATHO ULLAH,17/04/2025,41,18,2932,39,17.84


Dataset dimuat dari file CSV yang berisi data kebugaran jasmani dari beberapa individu. Data akan diproses lebih lanjut untuk memahami struktur dan isinya.

# Data Preparation & Preprocessing

In [3]:
# Konversi kolom waktu
df['BULAN'] = pd.to_datetime(df['BULAN'], dayfirst=True)
df = df.sort_values('BULAN')
df.set_index('BULAN', inplace=True)

# Ambil kolom numerik (kecuali ID & NAMA)
numeric_cols = df.columns.difference(['ID', 'NAMA'])
df_numeric = df[numeric_cols]

# Normalisasi dan konversi format angka
scaler = MinMaxScaler()
for col in numeric_cols:
    df_numeric[col] = df_numeric[col].astype(str).str.replace(',', '.', regex=False)
    df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')

df_numeric.dropna(inplace=True)
scaled_data = scaler.fit_transform(df_numeric)


<ipython-input-3-561683427>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric[col] = df_numeric[col].astype(str).str.replace(',', '.', regex=False)
<ipython-input-3-561683427>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
<ipython-input-3-561683427>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

Data dibersihkan dan disiapkan untuk model:

- Kolom waktu diubah menjadi format datetime dan dijadikan index.

- Kolom numerik dipilih dan dikonversi dari string ke float.

- Nilai hilang dihapus.

- Data dinormalisasi agar semua fitur berada pada skala yang sama (0–1) menggunakan MinMaxScaler.

 # Dataset Construction for LSTM

In [4]:
def create_dataset(data, window_size=2):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 2
X, y = create_dataset(scaled_data, window_size)


# Model Building & Training

In [5]:
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    Dense(df_numeric.shape[1])
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Prediction: Semua Data (Global Forecast)

In [6]:
# Ambil data paling akhir untuk prediksi selanjutnya
last_sequence = scaled_data[-window_size:]
last_sequence = last_sequence.reshape((1, window_size, df_numeric.shape[1]))

next_month_pred = model.predict(last_sequence)
next_month_pred = scaler.inverse_transform(next_month_pred)[0]

# Ambil ID terakhir (opsional)
last_id = df['ID'].iloc[-1] if 'ID' in df.columns else 'UNKNOWN'

# Tampilkan hasil prediksi
predicted_df = pd.DataFrame([next_month_pred], columns=df_numeric.columns)
predicted_df.insert(0, 'ID', last_id)

print("Prediksi bulan berikutnya:")
print(predicted_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
Prediksi bulan berikutnya:
     ID  LARI 12 MENIT    PULL UP    PUSH UP  SHUTTLE RUN   SIT UP 
0  P500    2447.676758  10.705875  32.378471    15.169398  35.36993


Model digunakan untuk memprediksi data bulan selanjutnya berdasarkan urutan data terakhir dari seluruh dataset.
Prediksi di-inverse transform untuk mengembalikan data ke skala aslinya.

# Prediction Berdasarkan ID yang Dipilih

In [7]:
input_id = input("Masukkan ID yang ingin diprediksi: ")
df_id = df[df['ID'] == input_id].copy()

if len(df_id) <= window_size:
    print(f"❌ Data untuk ID '{input_id}' tidak cukup untuk prediksi (minimal {window_size+1} baris).")
else:
    df_id = df_id.sort_index()
    df_id_numeric = df_id[numeric_cols]

    for col in df_id_numeric.columns:
        df_id_numeric[col] = df_id_numeric[col].astype(str).str.replace(',', '.', regex=False)
        df_id_numeric[col] = pd.to_numeric(df_id_numeric[col], errors='coerce')

    df_id_numeric.dropna(inplace=True)
    scaled_id_data = scaler.transform(df_id_numeric)

    last_sequence = scaled_id_data[-window_size:]
    last_sequence = last_sequence.reshape((1, window_size, df_id_numeric.shape[1]))

    pred_scaled = model.predict(last_sequence)
    pred = scaler.inverse_transform(pred_scaled)[0]

    predicted_df = pd.DataFrame([pred], columns=df_id_numeric.columns)
    predicted_df.insert(0, 'ID', input_id)

    print(f"\n📈 Prediksi bulan berikutnya untuk ID '{input_id}':")
    print(predicted_df)


Masukkan ID yang ingin diprediksi: P001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step

📈 Prediksi bulan berikutnya untuk ID 'P001':
     ID  LARI 12 MENIT    PULL UP    PUSH UP  SHUTTLE RUN    SIT UP 
0  P001     2446.34668  10.610976  33.597641    16.687588  35.953583


<ipython-input-7-740174216>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_numeric[col] = df_id_numeric[col].astype(str).str.replace(',', '.', regex=False)
<ipython-input-7-740174216>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_numeric[col] = pd.to_numeric(df_id_numeric[col], errors='coerce')
<ipython-input-7-740174216>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

# Prediction Berdasarkan Input Manual Terbaru

In [8]:
input_id = input("Masukkan ID yang ingin digunakan sebagai dasar prediksi: ")
df_id = df[df['ID'] == input_id].copy()
df_id = df_id.sort_index()

if len(df_id) < window_size - 1:
    print(f"❌ Data untuk ID '{input_id}' tidak cukup untuk ambil histori {window_size - 1} bulan.")
else:
    df_id_numeric = df_id[numeric_cols]
    for col in df_id_numeric.columns:
        df_id_numeric[col] = df_id_numeric[col].astype(str).str.replace(',', '.', regex=False)
        df_id_numeric[col] = pd.to_numeric(df_id_numeric[col], errors='coerce')

    df_id_numeric.dropna(inplace=True)

    base_sequence = df_id_numeric[-(window_size - 1):].values.tolist()

    print(f"\n📝 Masukkan data bulan terakhir (manual):")
    new_data = []
    for col in numeric_cols:
        val = input(f"  {col}: ").replace(',', '.')
        new_data.append(float(val))

    full_sequence = base_sequence + [new_data]
    full_sequence = np.array(full_sequence)

    scaled_sequence = scaler.transform(full_sequence)
    scaled_sequence = scaled_sequence.reshape((1, window_size, len(numeric_cols)))

    pred_scaled = model.predict(scaled_sequence)
    pred = scaler.inverse_transform(pred_scaled)[0]

    predicted_df = pd.DataFrame([pred], columns=numeric_cols)
    predicted_df.insert(0, 'ID', input_id)

    print(f"\n📈 Prediksi bulan selanjutnya untuk ID '{input_id}' berdasarkan input terbaru:")
    print(predicted_df)


Masukkan ID yang ingin digunakan sebagai dasar prediksi: P001


<ipython-input-8-3596442996>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_numeric[col] = df_id_numeric[col].astype(str).str.replace(',', '.', regex=False)
<ipython-input-8-3596442996>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_numeric[col] = pd.to_numeric(df_id_numeric[col], errors='coerce')
<ipython-input-8-3596442996>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea


📝 Masukkan data bulan terakhir (manual):
  LARI 12 MENIT: 2600
  PULL UP: 12
  PUSH UP: 30
  SHUTTLE RUN: 17
  SIT UP : 43
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

📈 Prediksi bulan selanjutnya untuk ID 'P001' berdasarkan input terbaru:
     ID  LARI 12 MENIT   PULL UP    PUSH UP  SHUTTLE RUN    SIT UP 
0  P001    2429.417236  9.363648  29.896917    15.451129  34.410591


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Fitur lanjutan: pengguna dapat memasukkan data bulan terakhir secara manual, dan sistem akan memprediksi bulan berikutnya berdasarkan input ini serta histori sebelumnya dari ID yang dipilih.

In [13]:
from google.colab import files
import shutil

# Zip folder agar bisa didownload
shutil.make_archive("tfjs_model", 'zip', "tfjs_model")
files.download("tfjs_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
import json

scaler_data = {
    "min": scaler.data_min_.tolist(),
    "max": scaler.data_max_.tolist(),
    "scale": scaler.scale_.tolist(),
    "min_": scaler.min_.tolist()
}

with open("scaler_lstm.json", "w") as f:
    json.dump(scaler_data, f)

files.download("scaler_lstm.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>